In [1]:
import time

from library_data import *
import library_models as lib
from library_models import *

In [2]:
import pandas as pd
from torch.nn import TransformerEncoderLayer, TransformerEncoder

In [3]:
class Args:
    def __init__(self):
        self.network = 'wikipedia'
        self.model = 'jodie'
        self.gpu = '0'
        self.epochs = 50
        self.embedding_dim = 128
        self.train_proportion = 0.8
        self.state_change = True
        self.datapath = "data/%s.csv" % self.network
args = Args()

In [4]:
[user2id, user_sequence_id, user_timediffs_sequence, user_previous_itemid_sequence,
 item2id, item_sequence_id, item_timediffs_sequence, 
 timestamp_sequence, feature_sequence, y_true] = load_network(args)



**** Loading wikipedia network from file: data/wikipedia.csv ****
Formating item sequence
Formating user sequence
Scaling timestamps
*** Network loading completed ***




In [5]:
data = pd.concat([pd.DataFrame([user_sequence_id, user_timediffs_sequence, 
                                item_sequence_id, item_timediffs_sequence, 
                                timestamp_sequence, y_true], 
                        index=["user_sequence_id", "user_timediffs_sequence", 
                               "item_sequence_id", "item_timediffs_sequence",
                               "timestamp_sequence", "y_true"]).T, 
                  pd.DataFrame(feature_sequence)], axis=1)
data

,user_sequence_id,user_timediffs_sequence,item_sequence_id,item_timediffs_sequence,timestamp_sequence,y_true,0,1,2,3,...,162,163,164,165,166,167,168,169,170,171
0,0.0,-0.273259,0.0,-0.190685,0.0,0.0,-0.175063,-0.176678,-0.937091,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
1,1.0,-0.273146,1.0,-0.190233,36.0,0.0,-0.175063,-0.176678,-0.937091,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
2,1.0,-0.273131,1.0,-0.190170,77.0,0.0,-0.175063,-0.176678,-0.937091,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
3,2.0,-0.272850,2.0,-0.189040,131.0,0.0,-0.175063,-0.176678,-0.937091,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
4,1.0,-0.273031,1.0,-0.189768,150.0,0.0,-0.175063,-0.176678,-0.937091,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157469,2003.0,-0.272647,632.0,-0.188224,2678155.0,0.0,0.090092,0.114777,0.810485,0.439210,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
157470,3762.0,-0.269217,798.0,-0.174425,2678158.0,0.0,-0.155317,-0.154974,1.263928,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
157471,2399.0,-0.272263,495.0,-0.186679,2678293.0,0.0,0.273444,0.300811,1.122434,-0.044391,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
157472,7479.0,-0.081557,920.0,0.564259,2678333.0,0.0,-0.175063,-0.176678,-0.937091,-0.381926,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775


In [ ]:
user_timestamp = min(data["user_timediffs_sequence"])
item_timestamp = min(data["item_timediffs_sequence"])
print(user_timestamp, item_timestamp)

In [ ]:
user_data = data.drop(["item_sequence_id"], axis=1)
user_data_sorted = user_data.sort_values(by="user_sequence_id")
user_data_padded = user_data_sorted.assign(key2=user_data_sorted.groupby('user_sequence_id').cumcount()).set_index(['user_sequence_id','key2']).unstack(fill_value=0).stack().reset_index('user_sequence_id')
user_data_padded_sorted = user_data_padded.sort_values(by=["user_sequence_id", "timestamp_sequence", "user_timediffs_sequence"], ascending=[True, True, False]).reset_index(drop=True)
user_data_padded_sorted.loc[user_data_padded_sorted["timestamp_sequence"] == 0, "user_timediffs_sequence"] = user_timestamp
user_data_padded_sorted.loc[user_data_padded_sorted["timestamp_sequence"] == 0, "item_timediffs_sequence"] = item_timestamp
user_data_reshape = user_data_padded_sorted.to_numpy().reshape(len(user2id), -1, 177)
user_data_padded_sorted

In [ ]:
item_data = data.drop(["user_sequence_id"], axis=1)
item_data_sorted = item_data.sort_values(by="item_sequence_id")
item_data_padded = item_data_sorted.assign(key2=item_data_sorted.groupby('item_sequence_id').cumcount()).set_index(['item_sequence_id','key2']).unstack(fill_value=0).stack().reset_index('item_sequence_id')
item_data_padded_sorted = item_data_padded.sort_values(by=["item_sequence_id", "timestamp_sequence", "item_timediffs_sequence"], ascending=[True, True, False]).reset_index(drop=True)
item_data_padded_sorted.loc[item_data_padded_sorted["timestamp_sequence"] == 0, "user_timediffs_sequence"] = user_timestamp
item_data_padded_sorted.loc[item_data_padded_sorted["timestamp_sequence"] == 0, "item_timediffs_sequence"] = item_timestamp
item_data_reshape = item_data_padded_sorted.to_numpy().reshape(len(item2id), -1, 177)
item_data_padded_sorted

In [ ]:
item_data_reshape.shape